In [196]:
import pandas as pd
your_games = pd.read_csv("your_games.csv")
all_games_details = pd.read_csv("final_games_details.csv")

In [197]:
# Merge the DataFrames on the gameID column
played_games_details = pd.merge(all_games_details, your_games[['gameID', 'review']], on='gameID', how='inner')

# Display the merged DataFrame
print("\nMerged DataFrame:")
print(played_games_details)



Merged DataFrame:
     gameID                                 Name  Positive  Negative  \
0         2                   Grand Theft Auto V   1171197    210154   
1        18                           ELDEN RING    460812     51238   
2        31                      DARK SOULS™ III    310407     21119   
3        33                             Portal 2    298300      3638   
4        37                        Borderlands 2    247833     16593   
..      ...                                  ...       ...       ...   
99     6869  Assassin’s Creed® Chronicles: India       499       271   
100    9574                 Hell is Other Demons       262        18   
101   10276                 Creature in the Well       229        64   
102   12343                           InnerSpace       156        93   
103   13453                                Aztez       129        36   

     Recommendations  Average playtime forever  Median playtime forever  \
0            1247051                     

In [198]:
all_devs = played_games_details['Dev'].str.split(',').explode().str.strip()
unique_devs = all_devs.unique()
print(unique_devs)
print(len(unique_devs))

['Rockstar Games' 'Rockstar North' 'Bandai Namco Entertainment'
 'FromSoftware Inc.' 'Inc.' 'FromSoftware' 'Valve' 'Aspyr (Linux)' '2K'
 'Gearbox Software' 'Aspyr (Mac)' 'Square Enix' 'Feral Interactive (Mac)'
 'Nixxes' 'Eidos-Montréal' 'Crystal Dynamics' 'Supergiant Games'
 'Activision (Excluding Japan and Asia)' 'FromSoftware (Japan)'
 '方块游戏 (Asia)' 'Ruffian Games' 'Splash Damage' 'Bungie'
 'Saber Interactive' 'Xbox Game Studios' '343 Industries' 'tobyfox'
 'Respawn Entertainment' 'Electronic Arts' 'Motion Twin'
 'Feral Interactive (Linux)' 'Rockstar Toronto' 'Massive Entertainment'
 'Ubisoft' 'and Ubisoft Shanghai' 'Ubisoft Montreal' '11 bit studios'
 'Inc' 'Deep Silver' '4A Games' 'QLOC' 'PlayStation PC LLC'
 'Santa Monica Studio' 'Quantic Dream' '3909' 'Lucas Pope' 'SEGA'
 'Konami Digital Entertainment' 'PlatinumGames' 'Avalanche Studios'
 'Bethesda Softworks' 'MachineGames' 'Asobo Studio' 'Focus Entertainment'
 'Playdead' 'DICE' '3D Realms' 'One More Level' 'Slipgate Ironworks™'


In [199]:
all_tags = played_games_details['full_tags'].str.split(',').explode().str.strip()

# Get unique values
unique_tags = all_tags.unique()
print(unique_tags)
print(len(unique_tags))

['Automobile Sim' 'Third-Person Shooter' 'Single-player' 'Atmospheric'
 'Comedy' 'Mature' 'Sandbox' 'Singleplayer' 'Steam Achievements'
 'Online Co-op' 'Moddable' 'Full controller support'
 'Remote Play on Tablet' 'Funny' 'Adventure' 'Crime' 'Co-op'
 'Remote Play on Phone' 'Great Soundtrack' 'Racing' 'PvP' 'Multi-player'
 'Action' 'Remote Play on TV' 'Multiplayer' 'Online PvP' 'Third Person'
 'Open World' 'Shooter' 'First-Person' 'Steam Trading Cards' 'Action RPG'
 'Fantasy' 'Difficult' 'Relaxing' 'Online Co-Op' 'Violent' 'Dark Fantasy'
 'Souls-like' 'Walking Simulator' '3D' 'RPG' 'Exploration' 'Replay Value'
 'Character Customization' 'Lore-Rich' 'Story Rich' 'Level Editor'
 'Female Protagonist' 'Puzzle' 'Sci-fi' 'Shared/Split Screen'
 'Remote Play Together' 'Stats' 'Commentary available' 'Physics'
 'Steam Workshop' 'Science' 'Includes level editor' 'Puzzle-Platformer'
 'Platformer' 'Dark Humor' 'Steam Cloud' 'FPS' 'Action-Adventure'
 'Shared/Split Screen Co-op' 'Captions available' '

In [200]:
import numpy as np
def getVector(gameID,data):
    row = data.loc[data['gameID'] == gameID].iloc[0]
    vector = []
    vector.append(row['Positive'])
    vector.append(row["Recommendations"])
    vector.append(row["rating"])
    vector.append(row["Average playtime forever"])
    vector.append(row["year"])
    game_devs = row['Dev'].split(",")
    dev_vector = [1 if unique_devs[i] in game_devs else 0 for i in range(len(unique_devs)) ]
    vector += dev_vector
    
    game_tags = row['full_tags'].split(",")
    tags_vector = [1 if unique_tags[i] in game_tags else 0 for i in range(len(unique_tags)) ]
    vector += tags_vector
    return np.array(vector)

In [201]:
def getX(data):
    vector_list = []
    played_games_id = data['gameID'].values.tolist()
    for gameID in played_games_id:
        vector_list.append(getVector(gameID,data))
    return vector_list

In [202]:
def getY(data):
    y = data['review'].values
    return y

In [203]:
#number of correct predictions / total predictions
def accuracy(pred, real):
    n = len(pred)
    number_of_correct_predictions = 0
    for i in range(n):
        if pred[i] == real[i]:
            number_of_correct_predictions += 1
    return round(number_of_correct_predictions * 100 / n, 2)

In [204]:
#true positive / all positive predictions
def precision(pred, real):
    n = len(pred)
    number_of_positive = 0
    number_of_true_positive = 0
    for i in range(n):
        if pred[i] == 1:
            number_of_positive += 1
            if real[i] == 1:
                number_of_true_positive += 1
    if number_of_positive == 0:
        return accuracy(pred, real)
    return round(number_of_true_positive * 100 / number_of_positive, 2)

In [205]:
# true positive / all positive instance
def recall(pred, real):
    n = len(pred)
    number_of_true_postive = 0
    number_of_real_positive = 0
    for i in range(n):
        if real[i] == 1:
            number_of_real_positive += 1
            if pred[i] == 1:
                number_of_true_postive += 1
    if (number_of_real_positive == 0):
        return accuracy(pred, real)
    return round(number_of_true_postive * 100 / number_of_real_positive, 2)

In [206]:
from sklearn.ensemble import RandomForestClassifier
def randomForest(X_train,X_test,Y_train,Y_test,number_of_trees = 1000, crite = 'gini', max_d = None, max_f = None, state = None):
    model = RandomForestClassifier(n_estimators= number_of_trees,criterion= crite, max_depth= max_d, max_features= max_f, random_state=state)
    model.fit(X_train, Y_train)
    Y_train_pred = model.predict(X_train)
    Y_pred = model.predict(X_test)
    return model, [accuracy(Y_pred,Y_test), precision(Y_pred,Y_test), recall(Y_pred,Y_test), accuracy(Y_train_pred,Y_train), precision(Y_train_pred,Y_train), recall(Y_train_pred,Y_train)]

In [207]:
def printMesure(m):
    print("Test accuracy:", m[0])
    print("Train accuracy:", m[3])
    print("Test precision:", m[1])
    print("Train precision:", m[4])
    print("Test recall:", m[2])
    print("Train recall:", m[5])

In [208]:
from sklearn.model_selection import train_test_split
def findBestModel(num_trees = 1000, rand_state = None, model_state = None):
    X = getX(played_games_details)
    Y = getY(played_games_details)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state= rand_state)
    crite_choices = ['gini', 'entropy', 'log_loss']
    features = ['sqrt', 'log2',None]
    choices = [[i,j,np.zeros(6)] for i in range(3) for j in range(3)]
    for i in range(3):
        for j in range(3):
            model, measure = randomForest(X_train,X_test,Y_train,Y_test,number_of_trees= num_trees, max_f=features[i], crite=crite_choices[j], state=model_state)
            print(crite_choices[j], features[i])
            print(measure)
            for choice in choices:
                if choice[0] == i and choice[1] == j:
                    choice[2] += measure
            print("---------------------------------------------------")
    choices = sorted(choices, key = lambda x: -sum(x[2][:3]))
    model, measure = randomForest(X,X,Y,Y,number_of_trees= num_trees, max_f=features[choices[0][0]], crite=crite_choices[choices[0][1]], state=model_state)
    print(choices)
    print(choices[0])
    print(crite_choices[choices[0][1]], features[choices[0][0]])
    printMesure(choices[0][2])
    return model, choices[0][2]


In [209]:
model, measure = findBestModel(num_trees= 100)

gini sqrt
[71.43, 70.0, 100.0, 100.0, 100.0, 100.0]
---------------------------------------------------
entropy sqrt
[71.43, 70.0, 100.0, 100.0, 100.0, 100.0]
---------------------------------------------------
log_loss sqrt
[66.67, 68.42, 92.86, 100.0, 100.0, 100.0]
---------------------------------------------------
gini log2
[71.43, 70.0, 100.0, 100.0, 100.0, 100.0]
---------------------------------------------------
entropy log2
[71.43, 70.0, 100.0, 100.0, 100.0, 100.0]
---------------------------------------------------
log_loss log2
[71.43, 70.0, 100.0, 100.0, 100.0, 100.0]
---------------------------------------------------
gini None
[57.14, 64.71, 78.57, 100.0, 100.0, 100.0]
---------------------------------------------------
entropy None
[57.14, 64.71, 78.57, 100.0, 100.0, 100.0]
---------------------------------------------------
log_loss None
[57.14, 64.71, 78.57, 100.0, 100.0, 100.0]
---------------------------------------------------
[[0, 0, array([ 71.43,  70.  , 100.  , 

In [210]:
from sklearn.tree import _tree

def get_classification_depth(clf, X):
    average_depths = []
    print("Number of not played games:", len(X))
    position = 0
    for sample in X:
        position += 1
        print("Checking game depth number:", position)
        sample_depth = 0
        count = 0

        for estimator in clf.estimators_:
            tree = estimator.tree_
            node = 0
            current_depth = 0

            while tree.feature[node] != _tree.TREE_UNDEFINED:
                if sample[tree.feature[node]] <= tree.threshold[node]:
                    node = tree.children_left[node]
                else:
                    node = tree.children_right[node]
                current_depth += 1

            if estimator.predict([sample])[0] == 1:
                sample_depth += current_depth
                count += 1

        if count == 0:
            average_depths.append(None)
        else:
            average_depths.append(round(sample_depth / count))
    print(average_depths)
    return average_depths


In [211]:
def getRecommendations(model):
    not_played_games = all_games_details[~all_games_details['gameID'].isin(your_games['gameID'])].head(2000)
    print("here")
    not_played_games_ids = not_played_games['gameID'].to_list()
    print("here")
    not_played_game_vectors = getX(not_played_games)
    print("here")
    recommendation_result = model.predict(not_played_game_vectors)
    print('here')
    not_played_games_depths = get_classification_depth(model, not_played_game_vectors)
    print('here')
    recommended_id = []
    for i in range(len(recommendation_result)):
        if recommendation_result[i] == 1:
            recommended_id.append((not_played_games_ids[i], not_played_games_depths[i]))
    print("here")
    recommended_df = pd.DataFrame(recommended_id, columns=['gameID', 'Depth'])
    recommended_game = pd.merge(all_games_details, recommended_df, on='gameID')
    recommended_game = recommended_game[['gameID', 'Name', 'Depth', 'rating', 'year']]
    recommended_game = recommended_game.sort_values(by=['Depth', 'gameID'], ascending=[True, True])
    recommended_game.insert(0, 'Rank', range(1, len(recommended_game) + 1))
    print(recommended_game)
    return recommended_game
    

In [212]:
rcm = getRecommendations(model)

here
here
here
here
Number of not played games: 2000
Checking game depth number: 1
Checking game depth number: 2
Checking game depth number: 3
Checking game depth number: 4
Checking game depth number: 5
Checking game depth number: 6
Checking game depth number: 7
Checking game depth number: 8
Checking game depth number: 9
Checking game depth number: 10
Checking game depth number: 11
Checking game depth number: 12
Checking game depth number: 13
Checking game depth number: 14
Checking game depth number: 15
Checking game depth number: 16
Checking game depth number: 17
Checking game depth number: 18
Checking game depth number: 19
Checking game depth number: 20
Checking game depth number: 21
Checking game depth number: 22
Checking game depth number: 23
Checking game depth number: 24
Checking game depth number: 25
Checking game depth number: 26
Checking game depth number: 27
Checking game depth number: 28
Checking game depth number: 29
Checking game depth number: 30
Checking game depth number

In [213]:
print(rcm)

      Rank  gameID                              Name  Depth  rating  year
476      1     547  Marvel's Guardians of the Galaxy    4.0   95.12  2021
655      2     741          Spyro™ Reignited Trilogy    4.0   94.90  2019
747      3     839                               FEZ    4.0   92.46  2013
1032     4    1136                        Road 96 🛣️    4.0   91.74  2021
1309     5    1420                   Virtual Cottage    4.0   97.92  2020
...    ...     ...                               ...    ...     ...   ...
1920  1953    2044             The House of Da Vinci   12.0   89.39  2017
165   1954     191                         Muse Dash   13.0   93.75  2019
502   1955     576                              ICEY   13.0   90.96  2016
1484  1956    1598                  Glass Masquerade   13.0   97.16  2016
1390  1957    1501         月圆之夜 (Night of Full Moon)    NaN   88.16  2019

[1957 rows x 6 columns]


In [214]:
import tkinter as tk
from tkinter import scrolledtext
import pandas as pd

class DataFrameViewer:
    def __init__(self, root, dataframe):
        self.root = root
        self.root.title('DataFrame Viewer')

        # Create scrolled text widget with monospaced font
        self.txt = scrolledtext.ScrolledText(self.root, width=100, height=20, wrap=tk.NONE, font=("Courier", 10))
        self.txt.pack(expand=True, fill=tk.BOTH)

        # Format DataFrame to string with proper alignment and fixed column widths
        try:
            df_str = self.format_dataframe(dataframe)
            self.txt.insert(tk.END, df_str)
        except AttributeError:
            self.txt.insert(tk.END, "Invalid DataFrame")

        # Disable editing
        self.txt.configure(state='disabled')

    def format_dataframe(self, dataframe):
        # Define column widths
        col_widths = {
            'Rank': 5,
            'gameID': 10,
            'Name': 50,
            'Depth': 10,
            'rating': 10,
            'year':10
        }

        # Create formatted string
        header = "".join([f"{col:{col_widths[col]}}" for col in dataframe.columns]) + "\n"
        rows = "\n".join(
            "".join([f"{str(value):{col_widths[col]}}" for col, value in row.items()])
            for _, row in dataframe.iterrows()
        )

        return header + rows

def main():
    # Example DataFrame (replace with your actual DataFrame)
    root = tk.Tk()
    app = DataFrameViewer(root, rcm)
    root.mainloop()

if __name__ == "__main__":
    main()
